In [14]:
import csv
import random
from random import shuffle

class DataHandler:
    def __init__(self, filepath):
        self.filepath = filepath

    def read_csv(self):
        with open(self.filepath, 'r') as file:
            return [line.strip().split(',') for line in file.readlines()]

    def train_test_split(self, dataset, split_ratio=0.8):
        random.shuffle(dataset)
        split_index = int(len(dataset) * split_ratio)
        return dataset[:split_index], dataset[split_index:]

    def separate_features_labels(self, dataset):
        features = [row[:-1] for row in dataset]
        labels = [row[-1] for row in dataset]

        # Encode categorical features and labels
        features, labels = self.encode_categorical(features, labels)

        return features, labels

    def encode_categorical(self, features, labels):
        feature_encodings = [{} for _ in range(len(features[0]))]
        label_encoding = {}

        encoded_features = []
        for feature_row in features:
            encoded_row = []
            for i, value in enumerate(feature_row):
                if value not in feature_encodings[i]:
                    feature_encodings[i][value] = len(feature_encodings[i])
                encoded_row.append(feature_encodings[i][value])
            encoded_features.append(encoded_row)

        encoded_labels = []
        for label in labels:
            if label not in label_encoding:
                label_encoding[label] = len(label_encoding)
            encoded_labels.append(label_encoding[label])

        return encoded_features, encoded_labels

In [15]:
import math
class NaiveBayesClassifier:
    def __init__(self):
        self.means = {}
        self.stds = {}
        self.class_probabilities = {}

    def fit(self, X, y):
        self._calculate_class_probabilities(y)
        self._calculate_means_stds(X, y)

    def _calculate_class_probabilities(self, y):
        class_counts = {label: y.count(label) for label in set(y)}
        total_count = len(y)
        self.class_probabilities = {label: count / total_count for label, count in class_counts.items()}

    def _calculate_means_stds(self, X, y):
        for label in self.class_probabilities:
            label_features = [X[i] for i in range(len(X)) if y[i] == label]
            self.means[label] = [sum(f) / len(f) for f in zip(*label_features)]
            self.stds[label] = [math.sqrt(sum([(x - mean)**2 for x in f]) / len(f)) for mean, f in zip(self.means[label], zip(*label_features))]

    def predict_single(self, input_features):
        probabilities = {}
        for label, _ in self.means.items():
            probabilities[label] = self.class_probabilities[label]
            for i, feature in enumerate(input_features):
                probabilities[label] *= self._calculate_probability(feature, self.means[label][i], self.stds[label][i])
        return max(probabilities, key=probabilities.get)

    def _calculate_probability(self, x, mean, std):
        if std == 0:
            return 1 if x == mean else 0
        else:
            exponent = math.exp(-(math.pow(x-mean, 2) / (2*math.pow(std, 2))))
            return (1 / (math.sqrt(2*math.pi) * std)) * exponent

    def predict(self, X):
        return [self.predict_single(features) for features in X]

    def classification_report(self, y_true, y_pred):
        unique_labels = set(y_true)
        report = {}
        for label in unique_labels:
            tp = sum(1 for i in range(len(y_true)) if y_true[i] == label and y_pred[i] == label)
            fp = sum(1 for i in range(len(y_true)) if y_true[i] != label and y_pred[i] == label)
            fn = sum(1 for i in range(len(y_true)) if y_true[i] == label and y_pred[i] != label)
            tn = sum(1 for i in range(len(y_true)) if y_true[i] != label and y_pred[i] != label)

            precision = tp / (tp + fp) if tp + fp > 0 else 0
            recall = tp / (tp + fn) if tp + fn > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
            accuracy = (tp + tn) / len(y_true)

            report[label] = {
                'Precision': precision,
                'Recall': recall,
                'F1-score': f1,
                'Accuracy': accuracy
            }

        return report


In [16]:
class KNNClassifier:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _euclidean_distance(self, point1, point2):
        return math.sqrt(sum((x - y) ** 2 for x, y in zip(point1, point2)))

    def predict_single(self, input_features):
        distances = [(self._euclidean_distance(input_features, x_train), y_train) for x_train, y_train in zip(self.X_train, self.y_train)]
        distances.sort(key=lambda x: x[0])
        neighbors = [label for _, label in distances[:self.k]]
        return max(set(neighbors), key=neighbors.count)

    def predict(self, X):
        return [self.predict_single(features) for features in X]

    def classification_report(self, y_true, y_pred):
        unique_labels = set(y_true)
        report = {}
        for label in unique_labels:
            tp = sum(1 for i in range(len(y_true)) if y_true[i] == label and y_pred[i] == label)
            fp = sum(1 for i in range(len(y_true)) if y_true[i] != label and y_pred[i] == label)
            fn = sum(1 for i in range(len(y_true)) if y_true[i] == label and y_pred[i] != label)
            tn = sum(1 for i in range(len(y_true)) if y_true[i] != label and y_pred[i] != label)

            precision = tp / (tp + fp) if tp + fp > 0 else 0
            recall = tp / (tp + fn) if tp + fn > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
            accuracy = (tp + tn) / len(y_true)

            report[label] = {
                'Precision': precision,
                'Recall': recall,
                'F1-score': f1,
                'Accuracy': accuracy
            }

        return report

In [17]:
class SVMClassifier:
    def __init__(self, learning_rate=0.001, epochs=1000, lambda_param=0.01):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.lambda_param = lambda_param

    def fit(self, X, y):
        self.weights = [0.0] * len(X[0])
        self.bias = 0.0
        self.y = [-1 if i == 0 else 1 for i in y]

        for _ in range(self.epochs):
            for idx, x_i in enumerate(X):
                condition = self.y[idx] * (sum([w * x for w, x in zip(self.weights, x_i)]) + self.bias) >= 1
                if condition:
                    self.weights = [w - self.learning_rate * (2 * self.lambda_param * w) for w in self.weights]
                else:
                    self.weights = [w + self.learning_rate * (x_i[j] * self.y[idx] - 2 * self.lambda_param * w) for j, w in enumerate(self.weights)]
                    self.bias += self.learning_rate * self.y[idx]

    def predict_single(self, x):
        linear_output = sum([w * x_i for w, x_i in zip(self.weights, x)]) + self.bias
        return 1 if linear_output >= 0 else 0

    def predict(self, X):
        return [self.predict_single(x) for x in X]

    def classification_report(self, y_true, y_pred):
        unique_labels = set(y_true)
        report = {}
        for label in unique_labels:
            tp = sum(1 for i in range(len(y_true)) if y_true[i] == label and y_pred[i] == label)
            fp = sum(1 for i in range(len(y_true)) if y_true[i] != label and y_pred[i] == label)
            fn = sum(1 for i in range(len(y_true)) if y_true[i] == label and y_pred[i] != label)
            tn = sum(1 for i in range(len(y_true)) if y_true[i] != label and y_pred[i] != label)

            precision = tp / (tp + fp) if tp + fp > 0 else 0
            recall = tp / (tp + fn) if tp + fn > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
            accuracy = (tp + tn) / len(y_true)

            report[label] = {
                'Precision': precision,
                'Recall': recall,
                'F1-score': f1,
                'Accuracy': accuracy
            }

        return report

In [18]:
def main():
    filepath = '/content/Obesity Classification.csv'
    data_handler = DataHandler(filepath)
    dataset = data_handler.read_csv()
    train_set, test_set = data_handler.train_test_split(dataset)

    train_features, train_labels = data_handler.separate_features_labels(train_set)
    test_features, test_labels = data_handler.separate_features_labels(test_set)

    # Naive Bayes
    nb_classifier = NaiveBayesClassifier()
    nb_classifier.fit(train_features, train_labels)
    nb_predictions = nb_classifier.predict(test_features)
    nb_report = nb_classifier.classification_report(test_labels, nb_predictions)
    print("Naive Bayes Classification Report:")
    for label, metrics in nb_report.items():
        print(f"Class {label}:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value:.2f}")
        print()

    # K-Nearest Neighbors
    knn_classifier = KNNClassifier(k=3)
    knn_classifier.fit(train_features, train_labels)
    knn_predictions = knn_classifier.predict(test_features)
    knn_report = knn_classifier.classification_report(test_labels, knn_predictions)
    print("KNN Classification Report:")
    for label, metrics in knn_report.items():
        print(f"Class {label}:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value:.2f}")
        print()

    # Support Vector Machine
    svm_classifier = SVMClassifier()
    svm_classifier.fit(train_features, train_labels)
    svm_predictions = svm_classifier.predict(test_features)
    svm_report = svm_classifier.classification_report(test_labels, svm_predictions)
    print("SVM Classification Report:")
    for label, metrics in svm_report.items():
        print(f"Class {label}:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value:.2f}")
        print()

if __name__ == "__main__":
    main()

Naive Bayes Classification Report:
Class 0:
  Precision: 0.64
  Recall: 0.64
  F1-score: 0.64
  Accuracy: 0.64

Class 1:
  Precision: 0.38
  Recall: 0.60
  F1-score: 0.46
  Accuracy: 0.68

Class 2:
  Precision: 0.00
  Recall: 0.00
  F1-score: 0.00
  Accuracy: 0.82

Class 3:
  Precision: 0.00
  Recall: 0.00
  F1-score: 0.00
  Accuracy: 0.77

KNN Classification Report:
Class 0:
  Precision: 0.67
  Recall: 0.36
  F1-score: 0.47
  Accuracy: 0.59

Class 1:
  Precision: 0.43
  Recall: 0.60
  F1-score: 0.50
  Accuracy: 0.73

Class 2:
  Precision: 0.00
  Recall: 0.00
  F1-score: 0.00
  Accuracy: 0.64

Class 3:
  Precision: 0.00
  Recall: 0.00
  F1-score: 0.00
  Accuracy: 0.68

SVM Classification Report:
Class 0:
  Precision: 0.00
  Recall: 0.00
  F1-score: 0.00
  Accuracy: 0.45

Class 1:
  Precision: 0.24
  Recall: 1.00
  F1-score: 0.38
  Accuracy: 0.27

Class 2:
  Precision: 0.00
  Recall: 0.00
  F1-score: 0.00
  Accuracy: 0.82

Class 3:
  Precision: 0.00
  Recall: 0.00
  F1-score: 0.00
  Acc